In [139]:
import pandas as pd
import torch

data = pd.read_csv('data.csv')

In [140]:
one_hot = pd.get_dummies(data['Intent'])
data = data.drop('Intent', axis=1)
data = data.join(one_hot)

data.head()

,Example,Complaint,Farewell,Feedback,Greet,Inquiry,Navigation,Request
0,Hi,False,False,False,True,False,False,False
1,Hello,False,False,False,True,False,False,False
2,Hey there,False,False,False,True,False,False,False
3,Good morning,False,False,False,True,False,False,False
4,Howdy,False,False,False,True,False,False,False


In [141]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, train_size=0.8, random_state=10)
validation_data, test_data = train_test_split(test_data, train_size=0.5, random_state=10)

train_data.shape, test_data.shape, validation_data.shape

((292, 8), (37, 8), (36, 8))

In [142]:
# print the first 5 rows of the train_data
train_data.head()

,Example,Complaint,Farewell,Feedback,Greet,Inquiry,Navigation,Request
34,A warm welcome to class!,False,False,False,True,False,False,False
87,Until we meet again,False,True,False,False,False,False,False
57,Have a good one,False,True,False,False,False,False,False
152,How do I change my mailing address?,False,False,False,False,True,False,False
204,The product is defective.,True,False,False,False,False,False,False


In [143]:
# turn everything to lowercase
train_data['Example'] = train_data['Example'].str.lower()
test_data['Example'] = test_data['Example'].str.lower()
validation_data['Example'] = validation_data['Example'].str.lower()

In [144]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokens = tokenizer.tokenize(train_data['Example'].values[0])

tokenizer.convert_tokens_to_ids(tokens)

/Users/lakshithnishshanke/miniconda3/envs/labml/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[1037, 4010, 6160, 2000, 2465, 999]

In [145]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        sentences = data['Example'].values
        
        inputs = tokenizer(sentences.tolist(), padding=True, truncation=True, return_tensors='pt')['input_ids']
        
        self.inputs = inputs#torch.unsqueeze(inputs, 2)
            
        print(self.inputs.shape)
        
        self.outputs = torch.tensor(data.drop('Example', axis=1).values, dtype=torch.float32)

    def __getitem__(self, index):
        return self.inputs[index], self.outputs[index]

    def __len__(self):
        return len(self.inputs)

In [146]:
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)
validation_dataset = CustomDataset(validation_data)

torch.Size([292, 20])
torch.Size([37, 20])
torch.Size([36, 17])


In [147]:
batch_size = 16

In [148]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

len(train_loader), len(test_loader), len(validation_loader)

(19, 3, 3)

In [149]:
# print the first batch of the train_loader
next(iter(train_loader))

[tensor([[  101,  2129,  2079,  1045,  4638,  2026,  2924,  4070,  5703,  1029,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  2045,  2001,  1037,  6707,  1999,  2026,  3021,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  2064,  2017,  2393,  2033,  2424,  1037,  3460,  2040, 16997,
           1999,  2035,  2121, 17252,  1029,   102,     0,     0,     0,     0],
         [  101,  2156,  2017,  2574,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  1996,  5746,  3737,  2006,  1996,  2678,  2655,  2003,  2200,
           3532,  1012,   102,     0,     0,     0,     0,     0,     0,     0],
         [  101,  7632,  2129,  2064,  1045,  2393,  2017,  1029,   102,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  2064,  2057,  88

In [150]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.hidden_size = 64

        self.i2h = nn.Linear(1, self.hidden_size)
        self.h2h = nn.Linear(self.hidden_size, self.hidden_size)
        self.h2o = nn.Linear(self.hidden_size, 7)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 64
rnn = RNN()

rnn = rnn.float()

In [151]:
input = torch.tensor([1.0])
hidden = torch.zeros(1, 64)

output, next_hidden = rnn(input, hidden)
output

tensor([[-1.8955, -2.3766, -1.9786, -1.9308, -1.7577, -1.5661, -2.3827]],
       grad_fn=<LogSoftmaxBackward0>)

In [152]:
criterion = nn.NLLLoss()
learning_rate = 1e-4

In [153]:
import wandb

configs = {
    "learning_rate": learning_rate,
    "architecture": "RNN",
    "dataset": "custom",
}

wandb.init(
    project="Intellihack_2",
    config=configs
)

Accuracy,▂▂▇▇▇▇██▇████████▅▅▅▅▅▄▄▄▄▄▄▂▂▂▂▂▂▂▁▁▁▁▁
Loss,▇▇▆▆▆▇▅▆▆▇█▆▅▇▅▇▅▇▇▄▃▃▅▆▅▂▆▆▃▂▂▄▁▃▆▃▃▁▅▁
Accuracy,0.11538
Loss,1.5085


In [154]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    line_tensor = line_tensor.float()
    
    # print(torch.unsqueeze(line_tensor[:,0], 1).shape)

    rnn.zero_grad()

    for i in range(line_tensor.size(1)):
        output, hidden = rnn(torch.unsqueeze(line_tensor[:,i], 1), hidden)
    
    # print(output.shape, category_tensor.shape)
    
    # convert category tensor to indices
    category_tensor = torch.argmax(category_tensor, dim=1)
    
    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    wandb.log({
        "Loss": loss.item()
    })

    print(loss.item())

In [155]:
def test_evaluate():
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (input, output_data) in enumerate(test_loader):
            hidden = rnn.initHidden()
            input = input.float()
            
            for i in range(input.size(1)):
                output, hidden = rnn(torch.unsqueeze(input[:,i], 1), hidden)
            
            total += output.size(0)
            correct += (torch.argmax(output_data, dim=1) == torch.argmax(output, dim=1)).sum().item()
            
        wandb.log({
            "Accuracy": correct / total
        })
            
    print('Accuracy of the network on the test examples: %d %%' % (100 * correct / total))

In [156]:
# iterate data loader and train
epochs = 5000
for epoch in range(epochs):
    for i, (input, output) in enumerate(train_loader):
        train(output, input)
    test_evaluate()

1.922032356262207
1.9612518548965454
1.8977556228637695
1.9937801361083984
2.023128032684326
1.9965710639953613
2.037860870361328
1.9006940126419067
1.9814633131027222
1.930048942565918
1.9601843357086182
2.13179087638855
1.9846960306167603
1.9001103639602661
1.9204853773117065
1.9769169092178345
1.9082796573638916
1.9955825805664062
2.0865907669067383
Accuracy of the network on the test examples: 16 %
1.9519550800323486
1.9374430179595947
1.900606393814087
1.9617151021957397
1.9623208045959473
2.049387216567993
1.9414230585098267
2.0081543922424316
2.008598566055298
2.0052216053009033
1.9126200675964355
1.971724033355713
2.0224690437316895
1.9621108770370483
1.9383792877197266
1.937119960784912
1.9411978721618652
2.010361433029175
2.0586583614349365
Accuracy of the network on the test examples: 16 %
1.9708456993103027
1.9574612379074097
1.9517277479171753
2.0747463703155518
2.044070243835449
1.9600518941879272
1.9633036851882935
1.9422340393066406
1.9776912927627563
1.940463900566101


KeyboardInterrupt: 